In [1]:
# import os
# cwd = os.getcwd()
# pygad_loc = '/grid/home/nbourgeois/wip/codon_optimization'
# os.chdir(pygad_loc)
# import pygad
# import numpy
# import os
# import sys
# import Bio
# import pandas as pd
# from general_functions import *
# from metrics import *
# import hashlib
# import subprocess
# import time
# os.chdir(cwd)
import random

import os
import pygad

cwd = os.getcwd()
pygad_loc = '/grid/home/nbourgeois/codonOpt'
os.chdir(pygad_loc)
from general_functions import *
from metrics import *
os.chdir(cwd)
import seaborn as sns

Using TensorFlow backend.


In [2]:
tissue = 'Liver'
cai_weights = get_codon_weights(tissue)
bai_weights = get_bicodon_weights(tissue)
codon_usage_table_loc = os.path.join(pygad_loc,'references','codon_usage.getex.txt')

In [3]:
def get_back_table(codon_usage_table_loc=os.path.join(pygad_loc,'references/codon_usage.getex.txt')):
    codon_usage_table = pd.read_csv(codon_usage_table_loc,sep='\t')
    forward_table = pd.Series(codon_usage_table.AA.values,index=codon_usage_table.Codon).to_dict()

    back_table = {}
    for key in forward_table:
        val = forward_table[key]
        if val not in back_table.keys():
            back_table[val] = [key]
        else:
            back_table[val].append(key)

    codon_to_int = {}
    i=0
    for codon in forward_table.keys():
        codon_to_int[codon] = i
        codon_to_int[i] = codon
        i += 1
    return(back_table)

def get_codon_space(aa_seq,back_table):
    gene_space = []
    for aa in aa_seq:
        all_cds = back_table[aa]
        gene_space.append(all_cds)
    return(gene_space)

In [4]:
def gen_rand_aa(n):
    rand_aas = max(n-2,0)
    aa_list = ['A','R','N','D','C','Q','G','E','H','I',
               'L','K','F','P','S','T','W','Y','V']
    index_list = np.random.randint(19,size=rand_aas)
    aa_seq = 'M'+''.join([aa_list[ind] for ind in index_list]) + '*'
    return(aa_seq)



In [5]:
random.random()*19
random.getrandbits(2)

3

In [6]:
def gen_random_seqs(codon_space, num_seqs):
    positions = len(codon_space)
    size_vector = [len(codon_pos) for codon_pos in codon_space] 
    seqs = [None]*num_seqs
    for i in range(num_seqs):
#         rand_index = [random.randrange(size) for size in size_vector]
        rand_index = [int(size * random.random()) for size in size_vector]
        seqs[i] = ''.join(([space[ind] for ind, space in zip(rand_index,codon_space)]))
    return(seqs)



In [ ]:
%%timeit

aa_seq = gen_rand_aa(200)

codon_to_int, codon_space = init_parameters_fun(aa_seq,codon_usage_table_loc)
   
seqs = gen_random_seqs(codon_space,1000)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/data/software/anaconda3/envs/bfx-codon/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_100143/2624769001.py", line 1, in <module>
    get_ipython().run_cell_magic('timeit', '', '\n# aa_seq = gen_rand_aa(200)\n\n# codon_to_int, codon_space = init_parameters_fun(aa_seq,codon_usage_table_loc)\n   \nseqs = gen_random_seqs(codon_space,1000)\n')
  File "/data/software/anaconda3/envs/bfx-codon/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2403, in run_cell_magic
    result = fn(*args, **kwargs)
  File "/data/software/anaconda3/envs/bfx-codon/lib/python3.7/site-packages/decorator.py", line 232, in fun
    return caller(func, *(extras + args), **kw)
  File "/data/software/anaconda3/envs/bfx-codon/lib/python3.7/site-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File 

In [7]:
%%timeit
x='A'
x_list = [None]*100000
for i in range(100000):
    x_list[i] = x

3.42 ms ± 38.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
ind = 0
aa_len = 150
aa_max = 1500
iterator = 50
num_ntseqs = 1000
num_aaseqs = 10000
list_len = int(num_ntseqs*num_aaseqs*((aa_max-aa_len)/50 +1))
cais = [None]*list_len
bais = [None]*list_len
aa_seqs = [None]*list_len
nt_seqs = [None]*list_len
aa_lens = [None]*list_len

back_table = get_back_table(codon_usage_table_loc)
while(aa_len < aa_max+1):
    print(aa_len)
    for i in range(num_aaseqs):
        aa_seq = gen_rand_aa(aa_len)
        codon_space = get_codon_space(aa_seq,back_table)
        seqs = gen_random_seqs(codon_space,num_ntseqs)
        start = ind
        stop = int(num_ntseqs + ind + 1)
        aa_lens[start:stop] = [aa_len]*(num_ntseqs)
        aa_seqs[start:stop] = [aa_seq]*(num_ntseqs)
        cais[start:stop] = [get_cai(x,cai_weights) for x in seqs]
        bais[start:stop] = [get_bai(x,bai_weights) for x in seqs]
        nt_seqs[start:stop] = seqs
        ind+= num_ntseqs 
        

    aa_len+=iterator

150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000


In [ ]:
data = pd.DataFrame(data={'Length':aa_lens,'AA_seq':aa_seqs,'NT_seq':nt_seqs,'CAI':cais,'BAI':bais})
mean_df = data.groupby(['Length','AA_seq']).mean()

In [ ]:
mean_df

In [ ]:
sns.scatterplot(data=mean_df,x='Length',y='CAI')

In [ ]:
sns.scatterplot(data=mean_df,x='Length',y='BAI')

In [ ]:
mean_df2 = data.groupby(['Length']).mean()

In [ ]:
sns.scatterplot(data=mean_df2,x='Length',y='CAI')

In [ ]:
sns.scatterplot(data=mean_df2,x='Length',y='BAI')

In [ ]:
data.to_csv('random_seqs.csv')